In [1]:
import pandas as pd
from requests.exceptions import ChunkedEncodingError
import torch
import numpy as np

import dataprocessing as dp
import backend.config as config
import rnn
import util

/Users/maximilianrapp/PycharmProjects/nextgenbike/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
raw_data = ""
while raw_data == "":
    try:
        raw_data = dp.get_data_db()
    except ChunkedEncodingError:
        print("Couldn't get Data, retrying ...")

dataframe = pd.read_json(raw_data)
print(dataframe.head())

Couldn't get Data, retrying ...
Couldn't get Data, retrying ...
Couldn't get Data, retrying ...
Couldn't get Data, retrying ...
Couldn't get Data, retrying ...
Couldn't get Data, retrying ...
Couldn't get Data, retrying ...
Couldn't get Data, retrying ...
Couldn't get Data, retrying ...
Couldn't get Data, retrying ...
Couldn't get Data, retrying ...
Couldn't get Data, retrying ...


/var/folders/q4/b_qkk2qx16zdglt42g7c0dlr0000gn/T/ipykernel_86441/2652575126.py:8: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dataframe = pd.read_json(raw_data)


Asphalt Data points:  1257
Pavement Data points:  752
Gravel Data points:  0
Grass Data points:  886

Crash Data points:  0
                     time  trip_id  vibration  latitude  longitude  \
0 2023-11-05 17:35:18.484        0          0       0.0        0.0   
1 2023-11-05 17:35:18.537        0          0       0.0        0.0   
2 2023-11-05 17:35:18.589        0          0       0.0        0.0   
3 2023-11-05 17:35:18.641        0          0       0.0        0.0   
4 2023-11-05 17:35:24.245        0          0       0.0        0.0   

   acceleration_x  acceleration_y  acceleration_z  gyroscope_x  gyroscope_y  \
0             260              15             -53           51           54   
1             259              13             -56           16           22   
2             262               6             -57          -20           78   
3             255               8             -54            1          -11   
4             258               9             -58          -

In [3]:
df['time_second'] = df.time.map(lambda x: pd.Timestamp(x).floor(freq='S'))
df['time'] = df.time.map(pd.Timestamp.timestamp)

grouped = dataframe.groupby([dataframe.trip_id, dataframe.time_second])  # grouped.get_group(1)
x = []

for i, (trip_seconds, table) in enumerate(grouped):
    if (i + 1) % 100 == 0:
        print("# trip seconds: " + str(i + 1))

    train_input = table.drop(columns=['terrain', 'trip_id', 'crash', 'time_second', 'latitude', 'longitude'])
    if len(train_input.columns) != config.n_training_cols:
        raise Exception("Wrong number of input columns (should be)", config.n_training_cols,
                        ", please check data.\n"
                        "But input was: ", train_input)

    train_input = train_input.to_numpy()

    input_length = len(train_input)
    if input_length > config.batch_size:
        train_input = train_input[:config.batch_size]
    else:
        n_missing_rows = config.batch_size - len(train_input)
        for _ in range(n_missing_rows):
            fake_array = [1] * config.n_training_cols
            train_input = np.append(train_input, fake_array)

    x.append(train_input)

lat_list = grouped.latitude.mean()
print(lat_list)

AttributeError: 'DataFrame' object has no attribute 'time_second'